In [ ]:
from utils.scraper import apec_scraper, pole_emploi_scraper
from utils.entities import Annonce
from database.migration import insert_jobs

In [ ]:
jobs_apec = apec_scraper(nb_pages=1)
len(jobs_apec)

In [ ]:
jobs_pole_emploi = pole_emploi_scraper(nb_jobs=100)
len(jobs_pole_emploi)

In [ ]:
for i in Annonce.find_all(limit=1000):
    Annonce.delete(i.id)
insert_jobs(jobs_apec)
insert_jobs(jobs_pole_emploi)